# REDES NEURAIS ARTIFICIAIS
## Evaluate Improve e Tunning

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.8.0'

### Importar o dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [5]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
np.shape(X_train)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [41]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [42]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [43]:
ann.add(tf.keras.layers.Dense(units=6, activation='tanh'))

### Adding the third hidden layer

In [44]:
ann.add(tf.keras.layers.Dense(units=4, activation='tanh'))

### Adding the output layer

In [45]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [46]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [47]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 961us/step - loss: 0.5049 - accuracy: 0.7879
Epoch 2/100
250/250 [==============================] - 0s 952us/step - loss: 0.4625 - accuracy: 0.7964
Epoch 3/100
250/250 [==============================] - 0s 982us/step - loss: 0.4399 - accuracy: 0.8049
Epoch 4/100
250/250 [==============================] - 0s 972us/step - loss: 0.4264 - accuracy: 0.8142
Epoch 5/100
250/250 [==============================] - 0s 949us/step - loss: 0.4161 - accuracy: 0.8201
Epoch 6/100
250/250 [==============================] - 0s 977us/step - loss: 0.4077 - accuracy: 0.8265
Epoch 7/100
250/250 [==============================] - 0s 951us/step - loss: 0.4002 - accuracy: 0.8328
Epoch 8/100
250/250 [==============================] - 0s 948us/step - loss: 0.3937 - accuracy: 0.8385
Epoch 9/100
250/250 [==============================] - 0s 944us/step - loss: 0.3877 - accuracy: 0.8434
Epoch 10/100
250/250 [==============================] - 0s 951us/step - l

250/250 [==============================] - 0s 939us/step - loss: 0.3334 - accuracy: 0.8618
Epoch 81/100
250/250 [==============================] - 0s 936us/step - loss: 0.3331 - accuracy: 0.8625
Epoch 82/100
250/250 [==============================] - 0s 952us/step - loss: 0.3331 - accuracy: 0.8615
Epoch 83/100
250/250 [==============================] - 0s 990us/step - loss: 0.3328 - accuracy: 0.8625
Epoch 84/100
250/250 [==============================] - 0s 945us/step - loss: 0.3329 - accuracy: 0.8635
Epoch 85/100
250/250 [==============================] - 0s 969us/step - loss: 0.3328 - accuracy: 0.8625
Epoch 86/100
250/250 [==============================] - 0s 968us/step - loss: 0.3327 - accuracy: 0.8618
Epoch 87/100
250/250 [==============================] - 0s 935us/step - loss: 0.3325 - accuracy: 0.8618
Epoch 88/100
250/250 [==============================] - 0s 999us/step - loss: 0.3326 - accuracy: 0.8619
Epoch 89/100
250/250 [==============================] - 0s 960us/step - loss:

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [48]:
y_pred = ann.predict(X_test)
y_pred

array([[0.2718653 ],
       [0.32311898],
       [0.07407507],
       ...,
       [0.15312126],
       [0.20273384],
       [0.21292049]], dtype=float32)

In [49]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [50]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1508   87]
 [ 193  212]]


---
# Avaliando o Modelo

In [51]:
# Evaluating the ANN
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [52]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dense(units = 4, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [53]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

C:\Users\Chaydson\AppData\Local\Temp/ipykernel_10592/1297209367.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)


In [69]:
print(mean,variance)

0.83262500166893 0.010253815856542925


# Melhorando o modelo

In [67]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed
from tensorflow.keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 4, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [68]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

C:\Users\Chaydson\AppData\Local\Temp/ipykernel_10592/1297209367.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)


In [70]:
print(mean,variance)

0.83262500166893 0.010253815856542925


# Halving Grid Search

In [58]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 4, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size': [10, 25, 32],
    'epochs': [25, 50, 100],
    'optimizer': ['adam', 'rmsprop', 'adadelta']
}

halving_cv = HalvingGridSearchCV(estimator = classifier,
                                  param_grid = parameters,
                                  scoring="accuracy", 
                                  n_jobs=-1, 
                                  min_resources="exhaust", 
                                  factor=3,
                                  cv = 10)

C:\Users\Chaydson\AppData\Local\Temp/ipykernel_10592/732744305.py:19: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = build_classifier)


In [59]:
%%time
halving_cv = halving_cv.fit(X_train, y_train)

Epoch 1/50
320/320 [==============================] - 1s 992us/step - loss: 0.5691 - accuracy: 0.7947
Epoch 2/50
320/320 [==============================] - 0s 973us/step - loss: 0.4626 - accuracy: 0.7960
Epoch 3/50
320/320 [==============================] - 0s 981us/step - loss: 0.4339 - accuracy: 0.8062
Epoch 4/50
320/320 [==============================] - 0s 982us/step - loss: 0.4240 - accuracy: 0.8275
Epoch 5/50
320/320 [==============================] - 0s 978us/step - loss: 0.4231 - accuracy: 0.8267
Epoch 6/50
320/320 [==============================] - 0s 982us/step - loss: 0.4194 - accuracy: 0.8303
Epoch 7/50
320/320 [==============================] - 0s 974us/step - loss: 0.4182 - accuracy: 0.8285
Epoch 8/50
320/320 [==============================] - 0s 988us/step - loss: 0.4210 - accuracy: 0.8311
Epoch 9/50
320/320 [==============================] - 0s 979us/step - loss: 0.4155 - accuracy: 0.8309
Epoch 10/50
320/320 [==============================] - 0s 981us/step - loss: 0.418

In [71]:
print('Best params:\n',halving_cv.best_params_,'\nBest accuracy:\n',halving_cv.best_score_)

Best params:
 {'batch_size': 25, 'epochs': 50, 'optimizer': 'rmsprop'} 
Best accuracy:
 0.8357142857142857


In [72]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 4, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 25, epochs = 50)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

print(mean,variance)

C:\Users\Chaydson\AppData\Local\Temp/ipykernel_10592/3163501403.py:13: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = build_classifier, batch_size = 25, epochs = 50)


0.83299999833107 0.011363319667055014


#### Melhor média e variância: 0,

# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)